# Machine Learning Mastery With Python
## Chương 9-14: Xây dựng, Đánh giá & Tối ưu mô hình Machine Learning

Trong notebook này, chúng ta sẽ thực hiện các bước xây dựng, đánh giá và tối ưu hóa mô hình học máy:  
- Chia dữ liệu train/test  
- Đánh giá mô hình với validation set  
- So sánh nhiều thuật toán  
- Đánh giá bằng nhiều chỉ số  
- Tối ưu hyperparameter  
- Lưu & tải mô hình

---


## Mục lục
1. [Chia dữ liệu train/test](#split)
2. [Đánh giá mô hình với validation set (cross-validation)](#cv)
3. [So sánh nhiều thuật toán ML](#compare)
4. [Đánh giá bằng nhiều chỉ số (accuracy, confusion matrix, classification report)](#metrics)
5. [Tối ưu hyperparameter (GridSearchCV)](#tuning)
6. [Lưu và tải mô hình (pickle/joblib)](#save)
7. [Tổng kết](#summary)

<a id="split"></a>
## 1. Chia dữ liệu train/test

**Mục đích:**  
Chia dữ liệu thành 2 phần:  
- **Train set:** để huấn luyện mô hình  
- **Test set:** để kiểm tra mô hình trên dữ liệu chưa từng thấy

**Tỉ lệ phổ biến:** 70-80% train, 20-30% test

In [ ]:
from sklearn.model_selection import train_test_split

# Giả sử X, y đã chuẩn bị từ trước
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

print("Kích thước tập train:", X_train.shape)
print("Kích thước tập test:", X_test.shape)

<a id="cv"></a>
## 2. Đánh giá mô hình với validation set (Cross-Validation)

**Mục đích:**  
- Đánh giá mô hình khách quan hơn bằng cách chia nhỏ tập train thành nhiều phần (folds), lần lượt train và test trên từng phần.
- Phổ biến nhất là **K-Fold Cross-Validation** (thường dùng k=10).

**Ưu điểm:**  
- Giảm nguy cơ overfitting do chia dữ liệu ngẫu nhiên một lần.

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')

print("Độ chính xác trung bình (mean accuracy):", results.mean())
print("Độ lệch chuẩn:", results.std())

<a id="compare"></a>
## 3. So sánh nhiều thuật toán ML

**Mục đích:**  
- Thử nghiệm nhiều thuật toán khác nhau để xem cái nào phù hợp nhất với dữ liệu của bạn.
- Có thể thử Logistic Regression, KNN, Decision Tree, SVM, Naive Bayes, Random Forest, v.v.

**Lưu ý:**  
- Nên dùng cùng một quy trình cross-validation để so sánh công bằng.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

models = []
models.append(('LR', LogisticRegression(max_iter=1000)))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))

results = []
names = []

for name, model in models:
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    print(f"{name}: Mean={cv_results.mean():.3f}, Std={cv_results.std():.3f} :3")

### Trực quan hóa so sánh các thuật toán

Vẽ boxplot để so sánh trực quan hiệu năng các mô hình.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))
plt.boxplot(results, labels=names)
plt.title("So sánh độ chính xác các thuật toán ML")
plt.ylabel("Accuracy")
plt.show()

<a id="metrics"></a>
## 4. Đánh giá bằng nhiều chỉ số

**Mục đích:**  
- Không chỉ dùng accuracy, nên xem thêm confusion matrix, precision, recall, f1-score.

**Lưu ý:**  
- Đánh giá trên tập test, sau khi đã chọn model tốt nhất ở bước trên.

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Giả sử chọn Logistic Regression là model tốt nhất
best_model = LogisticRegression(max_iter=1000)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

print("Accuracy trên tập test:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

<a id="tuning"></a>
## 5. Tối ưu hyperparameter (GridSearchCV)

**Mục đích:**  
- Thử nghiệm nhiều giá trị tham số để tìm ra bộ tham số tốt nhất cho mô hình.

**Ví dụ:**  
- Tối ưu số láng giềng (n_neighbors) cho KNN

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

param_grid = {'n_neighbors': np.arange(1, 21)}
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

print("Best score:", grid.best_score_)
print("Best params:", grid.best_params_)

<a id="save"></a>
## 6. Lưu và tải mô hình (pickle/joblib)

**Mục đích:**  
- Lưu mô hình đã train để sử dụng lại mà không cần train lại từ đầu.

**Lưu ý:**  
- Có thể dùng pickle hoặc joblib

In [ ]:
import pickle

# Lưu mô hình
with open('best_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

# Tải lại mô hình
with open('best_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

# Kiểm tra mô hình đã tải
print("Dự đoán với model đã load:", loaded_model.predict(X_test[:5]))

<a id="summary"></a>
## 7. Tổng kết

- Đã thực hiện đầy đủ quy trình xây dựng, đánh giá và tối ưu mô hình học máy.
- So sánh nhiều thuật toán, chọn ra mô hình tốt nhất.
- Đánh giá với nhiều chỉ số, tối ưu tham số, lưu mô hình để tái sử dụng.